In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"vmukund36","key":"06d4b84021039475b1871727a1637c22"}'}

In [2]:
!ls -lha kaggle.json
!pip install -q kaggle # installing the kaggle package
!mkdir -p ~/.kaggle # creating .kaggle folder where the key should be placed
!cp kaggle.json ~/.kaggle/ # move the key to the folder
!pwd # checking the present working directory

-rw-r--r-- 1 root root 65 Sep 16 07:22 kaggle.json
/content


In [3]:
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d snehaanbhawal/resume-dataset

 94% 59.0M/62.5M [00:02<00:00, 32.3MB/s]
100% 62.5M/62.5M [00:02<00:00, 27.5MB/s]


In [5]:
!unzip /content/resume-dataset.zip

Archive:  /content/resume-dataset.zip
  inflating: Resume/Resume.csv       
  inflating: data/data/ACCOUNTANT/10554236.pdf  
  inflating: data/data/ACCOUNTANT/10674770.pdf  
  inflating: data/data/ACCOUNTANT/11163645.pdf  
  inflating: data/data/ACCOUNTANT/11759079.pdf  
  inflating: data/data/ACCOUNTANT/12065211.pdf  
  inflating: data/data/ACCOUNTANT/12202337.pdf  
  inflating: data/data/ACCOUNTANT/12338274.pdf  
  inflating: data/data/ACCOUNTANT/12442909.pdf  
  inflating: data/data/ACCOUNTANT/12780508.pdf  
  inflating: data/data/ACCOUNTANT/12802330.pdf  
  inflating: data/data/ACCOUNTANT/13072019.pdf  
  inflating: data/data/ACCOUNTANT/13130984.pdf  
  inflating: data/data/ACCOUNTANT/13294301.pdf  
  inflating: data/data/ACCOUNTANT/13491889.pdf  
  inflating: data/data/ACCOUNTANT/13701259.pdf  
  inflating: data/data/ACCOUNTANT/14055988.pdf  
  inflating: data/data/ACCOUNTANT/14126433.pdf  
  inflating: data/data/ACCOUNTANT/14224370.pdf  
  inflating: data/data/ACCOUNTANT/14449423

In [6]:
pip install -r /content/Requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.9/106.9 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 8.6 MB/s eta 0:

In [17]:
import os
import PyPDF2
import textract
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
from datasets import load_dataset
import re

# Load the Hugging Face job descriptions dataset
job_descriptions = load_dataset("jacob-hugging-face/job-descriptions")

# Select 10-15 job descriptions
job_descriptions = job_descriptions['train']['job_description'][:15]

# Initialize DistilBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased")

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_path):
    try:
        text = textract.process(pdf_path).decode("utf-8")
    except Exception:
        text = ""
    return text

# Preprocess text data
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text

# Define a function to calculate similarity between CV and job descriptions
def calculate_similarity(cv_embedding, job_description_embedding):
    similarity_scores = cosine_similarity(cv_embedding.reshape(1, -1), job_description_embedding.reshape(1, -1))
    return similarity_scores[0][0]

# Specify the folder containing CVs in PDF format
cv_folder = "/content/data/data/ACCOUNTANT"  # Replace with the path to your CVs

# List to store the top 5 CVs for each job description
top_cv_matches = []

# Loop through each job description
for job_description in job_descriptions:
    job_description = preprocess_text(job_description)

    # Encode the job description
    job_description_encoding = tokenizer.encode(job_description, return_tensors="pt", truncation=True, padding='max_length', max_length=512)

    # Calculate embeddings for the job description
    job_description_embedding = model(job_description_encoding).last_hidden_state.mean(dim=1).detach().numpy()

    # Dictionary to store CVs and their similarity scores
    cv_similarities = {}

    # Loop through CVs in the folder
    for cv_file in os.listdir(cv_folder):
        cv_path = os.path.join(cv_folder, cv_file)
        cv_text = extract_text_from_pdf(cv_path)
        cv_text = preprocess_text(cv_text)

        # Encode the CV
        cv_encoding = tokenizer.encode(cv_text, return_tensors="pt", truncation=True, padding='max_length', max_length=512)

        # Calculate embeddings for the CV
        cv_embedding = model(cv_encoding).last_hidden_state.mean(dim=1).detach().numpy()

        # Calculate cosine similarity
        similarity_scores = calculate_similarity(cv_embedding, job_description_embedding)

        # Store the CV and its similarity score
        cv_similarities[cv_file] = similarity_scores

    # Sort CVs by similarity score and select the top 5
    top_cv_matches.append(
        {
            "job_description": job_description,
            "top_cv_matches": dict(sorted(cv_similarities.items(), key=lambda item: item[1], reverse=True)[:5])
        }
    )

# Print the top 5 CVs for each job description
for i, job_matches in enumerate(top_cv_matches):
    print(f"Job Description {i + 1}:\n{job_matches['job_description']}\n")
    print("Top 5 CV Matches:")
    for cv_file, similarity in job_matches['top_cv_matches'].items():
        print(f"CV: {cv_file}, Similarity Score: {similarity:.4f}")
    print("\n")


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Job Description 1:
minimum qualifications
bachelors degree or equivalent practical experience years of experience in saas or productivity tools businessexperience managing enterprise accounts with sales cycles
preferred qualifications
 years of experience building strategic business partnerships with enterprise customersability to work through and with a reseller ecosystem to scale the businessability to plan pitch and execute a territory business strategyability to build relationships and to deliver results in a crossfunctionalmatrixed environmentability to identify crosspromoting and uppromoting opportunities within the existing account baseexcellent account management writtenverbal communication strategic and analyticalthinking skills
about the job
as a member of the google cloud team you inspire leading companies schools and government agencies to work smarter with google tools like google workspace search and chrome you advocate the innovative power of our products to make organiz